In [1]:
import torch
from WeaponData_alexnet import WeaponData
from torchvision import models
import torch.nn as nn
from torch.nn import functional as F

In [2]:
dataset = WeaponData()

['banknot', 'karta płatnicza', 'nóż', 'pistolet', 'portfel', 'smartphone']


In [3]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.alexnet(pretrained=True).cuda()
model.fc = nn.Sequential(
               nn.Linear(2048, 1024),
               nn.ReLU(inplace=True),
               nn.Linear(1024,512),
               nn.ReLU(inplace=True),
               nn.Linear(512, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 32),
               nn.ReLU(inplace=True),
               nn.Linear(32, 6)).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters())

In [4]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
                data = dataset.trainloader
            else:
                model.eval()
                data = dataset.testloader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data:
                inputs = inputs.cuda()
                labels = labels.cuda()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data)
            epoch_acc = running_corrects.double() / len(data)

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [5]:
model_trained = train_model(model, criterion, optimizer, num_epochs=50 )
torch.save(model_trained, "./output_data/models/weapon_alexnet")

Epoch 1/50
----------
train loss: 207.5689, acc: 0.0000
validation loss: 205.3616, acc: 0.0000
Epoch 2/50
----------
train loss: 207.8567, acc: 0.0059
validation loss: 205.3616, acc: 0.0000
Epoch 3/50
----------
train loss: 208.3216, acc: 0.0000
validation loss: 205.3616, acc: 0.0000
Epoch 4/50
----------
train loss: 207.9453, acc: 0.0059
validation loss: 205.3616, acc: 0.0000
Epoch 5/50
----------
train loss: 207.8820, acc: 0.0020
validation loss: 205.3616, acc: 0.0000
Epoch 6/50
----------
train loss: 208.1369, acc: 0.0020
validation loss: 205.3616, acc: 0.0000
Epoch 7/50
----------
train loss: 208.2026, acc: 0.0059
validation loss: 205.3616, acc: 0.0000
Epoch 8/50
----------
train loss: 208.0219, acc: 0.0000
validation loss: 205.3616, acc: 0.0000
Epoch 9/50
----------
train loss: 208.1778, acc: 0.0020
validation loss: 205.3616, acc: 0.0000
Epoch 10/50
----------
train loss: 207.8640, acc: 0.0000
validation loss: 205.3616, acc: 0.0000
Epoch 11/50
----------
train loss: 207.5449, acc: